In [2]:
!pip install mesa
import mesa
import numpy as np
from IPython.display import Javascript

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.6 MB/s eta 0:00:00


In [3]:
def randomStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = np.random.choice(other_agents)
    return random_card, other_agent

def randomOwnerStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = -1
    # This will break if we add a draw pile and an agent knows the random_card is in the draw pile
    while other_agent == -1:
        other_agent = np.random.choice(random_card.potential_owners)
    return random_card, other_agent

In [ ]:
class Disjunction:
    def __init__(self, other_agent, cards):
        self.other_agent = other_agent
        self.cards = cards

In [15]:
def flatten(l):
    return [item for sublist in l for item in sublist]

class Hand:
    def __init__(self, model, agent, other_agents):
        self.model = model
        self.agent = agent
        self.other_agents = other_agents
        self.quartet_groups = [QuartetGroup(quartet_set.group_name, quartet_set.card_names, agent, other_agents)
                              for quartet_set in self.model.quartet_sets]

    def getCards(self, flat = False):
        cards = [quarted_group.getCards() for quarted_group in self.quartet_groups]
        if flat:
            cards = flatten(cards)
        return cards

    def addCard(self, new_card):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == new_card.group_name:
                quartet_group.addCard(new_card)
                return quartet_group.isComplete()

    def missingCards(self, flat = False, exclude_unowned_groups = False):
        missing_cards = [group.missingCards() for group in self.quartet_groups]
        if exclude_unowned_groups:
            new_missing_cards = []
            for missing_cards_group in missing_cards:
                if len(missing_cards_group) != 4:
                    new_missing_cards.append(missing_cards_group)
            missing_cards = new_missing_cards

        if flat:
            missing_cards = flatten(missing_cards)
        return missing_cards

    def getGroup(self, group_name):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == group_name:
                return quartet_group

    def getCard(self, asked_card):
        quartet_group = self.getGroup(asked_card.group_name)
        for card in quartet_group.cards:
            if card.card_name == asked_card.card_name:
                return card

    def giveCard(self, asked_card):
        card = self.getCard(asked_card)
        if card.owned:
            card.owned = False
            return True
        return False

    def updateAgentHasGroup(self, other_agent_id, group_name, excluded_card):
        quartet_group = self.getGroup(group_name)
        quartet_group.updateAgentHasGroup(other_agent_id, excluded_card)

    def updateAgentHasCard(self, agent_id, card):
        quartet_group = self.getGroup(card.group_name)
        quartet_group.updateAgentHasCard(agent_id, card)

    def updateAgentDoesNotHaveCard(self, agent_id, card):
        quartet_group = self.getGroup(card.group_name)
        quartet_group.updateAgentDoesNotHaveCard(agent_id, card)

class QuartetGroup:
    def __init__(self, group_name, card_names, agent = -1, other_agents = []):
        self.group_name = group_name
        self.card_names = card_names
        self.agent = agent
        self.other_agents = other_agents
        self.cards = [QuartetCard(group_name, card_name, self) for card_name in card_names]
        self.knowledge_per_agent = {key:[] for key in other_agents}

    def getCardObject(self, asked_card):
        for card in self.cards:
            if asked_card == card:
                return card

    def addCard(self, new_card):
        for card in self.cards:
            if new_card == card:
                card.owned = True
                card.potential_owners = [self.agent]

    def missingCards(self):
        current_cards = self.getCards()
        return [x for x in self.cards if x not in current_cards]

    def getCards(self):
        return [x for x in self.cards if x.owned]

    def isComplete(self):
        return np.all([x.owned for x in self.cards])

    # Get all the cards that a given agent might have
    def getPotentialCardsOfOtherAgent(self, other_agent, excluded_card):
        potential_cards = []
        for card in self.cards:
            if card.owned: continue
            if card == excluded_card: continue
            if other_agent in card.potential_owners:
                potential_cards.append(card)
        return potential_cards

    def updateAgentHasGroup(self, other_agent, excluded_card):
        # Get the potential cards the agent can have
        potential_cards = self.getPotentialCardsOfOtherAgent(other_agent, excluded_card)
        # print("As agent", self.agent, "I know agent", other_agent, "can have cards", potential_cards)
        if len(potential_cards) == 1:
            potential_cards[0].potential_owners = [other_agent]
        else:
            self.knowledge_per_agent[other_agent] = potential_cards
            print("As agent", self.agent, "I have this knowledge:", self.knowledge_per_agent)

    def updateAgentHasCard(self, agent_id, card):
        card_object = self.getCardObject(card)
        card_object.potential_owners = [agent_id]

    def updateAgentDoesNotHaveCard(self, agent_id, card):
        card_object = self.getCardObject(card)
        try:
            card_object.potential_owners.remove(agent_id)
        except ValueError:
            pass

    def __repr__(self):
        return str([card for card in self.cards])

class QuartetCard:
    def __init__(self, group_name, card_name, quartet_group, owned = False):
        self.group_name = group_name
        self.card_name = card_name
        self.quartet_group = quartet_group
        self.owned = owned
        self.potential_owners = [i for i in quartet_group.other_agents]
        #self.potential_owners.append(-1)

    def __eq__(self, other):
        if not isinstance(other, QuartetCard):
            return NotImplemented

        return self.group_name == other.group_name and self.card_name == other.card_name

    def __repr__(self):
        return self.group_name + "_" + self.card_name

    def __hash__(self):
        return hash((self.group_name, self.card_name))

class QuartetAgent(mesa.Agent):

    def __init__(self, unique_id, model, strategy):
        super().__init__(unique_id, model)
        self.other_agents = [i for i in range(self.model.num_agents) if i != self.unique_id]
        self.hand = Hand(model, unique_id, self.other_agents)
        self.score = 0
        self.strategy = strategy


    def step(self):
        if not self.model.running:
            return
        # print("I am agent", self.unique_id)
        # print("I currently have these cards:")
        # print(self.hand.getCards())
        # print("I currently do not have these cards:")
        # print(self.hand.missingCards())

        missing_cards = self.hand.missingCards(flat = True, exclude_unowned_groups = True)
        if len(missing_cards) == 0:
            self.model.skipTurn(self.unique_id)
            return

        asked_card, other_agent = self.strategy(self.hand, self.unique_id, self.other_agents)

        got_card = self.model.askForCard(self, other_agent, asked_card)
        if got_card:
            self.addCard(asked_card)
            # print("Now, I have these cards:")
            # print(self.hand.getCards())
            # print("Now I can go again!")
            self.step()

    def addCard(self, new_card):
        has_complete_set = self.hand.addCard(new_card)
        if has_complete_set:
            self.score += 1
            self.model.announceCompleteGroup(self.unique_id, new_card.group_name)

    def giveCard(self, card):
        return self.hand.giveCard(card)

    def updateAgentHasGroup(self, other_agent_id, group_name, excluded_card):
        self.hand.updateAgentHasGroup(other_agent_id, group_name, excluded_card)

    def updateAgentHasCard(self, agent_id, card):
        self.hand.updateAgentHasCard(agent_id, card)

    def updateAgentDoesNotHaveCard(self, asked_agent_id, card):
        self.hand.updateAgentDoesNotHaveCard(asked_agent_id, card)



class QuartetModel(mesa.Model):

    def __init__(self, N, strategies):
        self.running = True
        self.num_agents = N
        self.strategies = strategies
        self.schedule = mesa.time.BaseScheduler(self)
        self.quartet_sets = [QuartetGroup('a', ['a_1', 'a_2', 'a_3', 'a_4']),
                             QuartetGroup('b', ['b_1', 'b_2', 'b_3', 'b_4']),
                             QuartetGroup('c', ['c_1', 'c_2', 'c_3', 'c_4'])]
        self.completed_sets = 0
        self.agent_can_ask_owned_card = False
        self.print_announcements = True
        # Create agents
        for i in range(self.num_agents):
            a = QuartetAgent(i, self, self.strategies[i])
            self.schedule.add(a)

        all_cards = []
        for quartet_set in self.quartet_sets:
            for quartet_card in quartet_set.cards:
                all_cards.append(quartet_card)
        np.random.shuffle(all_cards)

        cards_per_agent = 4
        for i in range(cards_per_agent):
            for agent in self.schedule.agents:
                current_card = all_cards.pop()
                agent.addCard(current_card)

    def step(self):
        """Advance the model by one step."""
        self.schedule.step()

    def askForCard(self, asking_agent, asked_agent_id, card):
        asking_agent_id = asking_agent.unique_id
        if self.print_announcements:
            print("PA: Agent", asking_agent_id, "asks agent", asked_agent_id, "for card", card)
        asked_agent_object = self.schedule.agents[asked_agent_id]
        self.announceAgentHasGroup(asking_agent, card)
        has_card = asked_agent_object.giveCard(card)
        if has_card:
            if self.print_announcements:
                print("PA: Agent", asked_agent_id, "had the card and gave it to", asking_agent_id)
            self.announceAgentHasCard(asking_agent_id, card)
        else:
            if self.print_announcements:
                print("PA: Agent", asked_agent_id, "did not have the card.")
            self.announceAgentDoesNotHaveCard(asked_agent_id, card)
        print("")
        return has_card

    def announceAgentHasGroup(self, asking_agent, card):
        # The asking agent has complete knowledge over its cards, so we can skip this
        # This is always a successful announcement
        group_name = card.group_name
        other_agents = [agent for agent in self.schedule.agents if agent != asking_agent]
        excluded_card = None
        if not self.agent_can_ask_owned_card:
            excluded_card = card
        for agent in other_agents:
            agent.updateAgentHasGroup(asking_agent.unique_id, group_name, excluded_card)

    def announceAgentHasCard(self, asking_agent_id, card):
        other_agents = [agent for agent in self.schedule.agents if agent != asking_agent_id]
        for agent in other_agents:
            agent.updateAgentHasCard(asking_agent_id, card)

    def announceAgentDoesNotHaveCard(self, asked_agent_id, card):
        other_agents = [agent for agent in self.schedule.agents if agent != asked_agent_id]
        for agent in other_agents:
            agent.updateAgentDoesNotHaveCard(asked_agent_id, card)

    def announceCompleteGroup(self, agent, group_name):
        if self.print_announcements:
            print("PA: Agent", agent, "has completed the set", group_name)
        self.completed_sets += 1
        if len(self.quartet_sets) == self.completed_sets:
            self.finishGame()

    def skipTurn(self, agent):
        if self.print_announcements:
            print("PA: Agent", agent, "cannot ask for any card and skips this turn.")

    def finishGame(self):
        print("----------------------------------------------------------------------------")
        print("The game is finished! Here are the scores:")
        for agent in self.schedule.agents:
            print("Agent", agent.unique_id, "has score", agent.score)
        self.running = False

    def getScores(self):
        scores = []
        for agent in self.schedule.agents:
            scores.append(agent.score)
        return scores

In [16]:
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))

strategyNames = [randomStrategy, randomOwnerStrategy]

strategies = [randomOwnerStrategy, randomOwnerStrategy, randomOwnerStrategy]

total_scores = [0, 0, 0]
for i in range(1):
    model = QuartetModel(3, strategies)
    model.run_model()
    total_scores = np.add(total_scores, model.getScores())

print(total_scores)

<IPython.core.display.Javascript object>

PA: Agent 0 asks agent 2 for card c_c_1
As agent 1 I have this knowledge: {0: [c_c_2, c_c_4], 2: []}
As agent 2 I have this knowledge: {0: [c_c_3, c_c_4], 1: []}
PA: Agent 2 had the card and gave it to 0

PA: Agent 0 asks agent 1 for card c_c_2
As agent 1 I have this knowledge: {0: [c_c_1, c_c_4], 2: []}
As agent 2 I have this knowledge: {0: [c_c_1, c_c_3, c_c_4], 1: []}
PA: Agent 1 did not have the card.

PA: Agent 1 asks agent 0 for card c_c_4
PA: Agent 0 had the card and gave it to 1

PA: Agent 1 asks agent 0 for card c_c_2
As agent 0 I have this knowledge: {1: [c_c_3, c_c_4], 2: []}
As agent 2 I have this knowledge: {0: [c_c_1, c_c_3, c_c_4], 1: [c_c_3, c_c_4]}
PA: Agent 0 did not have the card.

PA: Agent 2 asks agent 1 for card a_a_1
As agent 1 I have this knowledge: {0: [], 2: [a_a_2, a_a_3, a_a_4]}
PA: Agent 1 did not have the card.

PA: Agent 0 asks agent 2 for card c_c_2
PA: Agent 2 had the card and gave it to 0

PA: Agent 0 asks agent 1 for card c_c_4
As agent 1 I have this 

In [56]:
print(total_scores)

[1796  687  517]
